<a href="https://colab.research.google.com/github/Alaaibrahim2/Alaa/blob/main/seq2seq_bahadanau_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
df = pd.read_csv("/content/ara_eng.txt",delimiter="\t",names=["eng","ar"])
df

,eng,ar
0,Hi.,مرحبًا.
1,Run!,اركض!
2,Help!,النجدة!
3,Jump!,اقفز!
4,Stop!,قف!
...,...,...
24633,rising voices promoting a more linguistically ...,شاركنا تحدي ابداع ميم بلغتك الام تعزيزا للتنوع...
24634,following last year s successful campaign we i...,استكمالا لنجاح حملة العام السابق ندعوكم للمشار...
24635,during last year s challenge we also met langu...,تعرفنا خلال تحدي العام الماضي على ابطال لغويين...
24636,to take part just follow the simple steps outl...,للمشاركة في التحدي اتبع الخطوات الموضحة على ال...


In [3]:
data = pd.read_csv('/content/ara_eng.txt', sep="\t", names=["target", "source"])

In [4]:
data

,target,source
0,Hi.,مرحبًا.
1,Run!,اركض!
2,Help!,النجدة!
3,Jump!,اقفز!
4,Stop!,قف!
...,...,...
24633,rising voices promoting a more linguistically ...,شاركنا تحدي ابداع ميم بلغتك الام تعزيزا للتنوع...
24634,following last year s successful campaign we i...,استكمالا لنجاح حملة العام السابق ندعوكم للمشار...
24635,during last year s challenge we also met langu...,تعرفنا خلال تحدي العام الماضي على ابطال لغويين...
24636,to take part just follow the simple steps outl...,للمشاركة في التحدي اتبع الخطوات الموضحة على ال...


In [5]:
import os
import re
import numpy as np
!pip install evaluate
from pathlib import Path

import pandas as pd
import torch
import evaluate


from torch import nn
from tqdm.notebook import tqdm
from tokenizers import Tokenizer
from tokenizers import normalizers
from tokenizers import pre_tokenizers
from tokenizers.models import WordLevel, WordPiece
from tokenizers.processors import TemplateProcessing

from torch.utils.data import DataLoader, Dataset, Subset

from tokenizers.normalizers import NFC, StripAccents, Strip, Lowercase
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordLevelTrainer, WordPieceTrainer

from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00


In [6]:
punc_re = re.compile(r"""[!"#$%&\'()*+,-./:;<=>?@[\\\]^_`{|}~،؟…«“\":\"…”]""")
def remove_punctation(text: str) -> str:
    return punc_re.sub(repl="", string=text)


diactircs_re = re.compile("[\u064B-\u0652]")
def remove_diactrics(text: str) -> str:
    # TODO: Implement this function using the diactrics_re
    return diactircs_re.sub(repl="", string=text)

def clean_text(text: str) -> str:
    return remove_punctation(remove_diactrics(text))

In [7]:
data["target"] = data["target"].map(clean_text)
data["source"] = data["source"].map(clean_text)


In [8]:
data["target"].duplicated(keep=False).sum()

3241

In [9]:
train, test = train_test_split(data)

In [10]:
normalizer = normalizers.Sequence([NFC(), StripAccents(), Lowercase(), Strip()])


In [11]:
post_processor = TemplateProcessing(
    single="[SOS] $A [EOS]",
    special_tokens=[("[SOS]", 2), ("[EOS]", 3)]
)

In [12]:
pre_tokenizer = pre_tokenizers.Sequence([Whitespace()])


In [13]:
ar_tokenizer = Tokenizer(model=WordLevel(unk_token="[OOV]"))
en_tokenizer = Tokenizer(model=WordLevel(unk_token="[OOV]"))

In [14]:
ar_tokenizer.normalizer = normalizer
ar_tokenizer.pre_tokenizer = pre_tokenizer
ar_tokenizer.post_processor = post_processor
ar_tokenizer.enable_padding(direction='left', pad_id=0, pad_token='[PAD]')

en_tokenizer.normalizer = normalizer
en_tokenizer.pre_tokenizer = pre_tokenizer
en_tokenizer.post_processor = post_processor
en_tokenizer.enable_padding(direction='right', pad_id=0, pad_token='[PAD]')

In [15]:
trainer = WordLevelTrainer(vocab_size=200000, show_progress=True, min_frequency=2,
                           special_tokens=["[PAD]", "[OOV]", "[SOS]", "[EOS]"]
                          )

In [16]:
ar_tokenizer.train_from_iterator(iterator=data["source"].tolist(), trainer=trainer)
en_tokenizer.train_from_iterator(iterator=data["target"].tolist(), trainer=trainer)

In [17]:
ar_tokenizer.get_vocab()


{'وفيميو': 24620,
 'والروابط': 24084,
 'والانتخابات': 12548,
 'واخبار': 16346,
 'فرصة': 813,
 'الجهاد': 18040,
 'بطبيب': 19756,
 'واد': 12512,
 'اشيباشي': 17466,
 'المقر': 11077,
 'مجيد': 5883,
 'التقت': 10769,
 'مراسل': 4836,
 'نتطلع': 7481,
 'الكردي': 13755,
 'زعزعة': 14993,
 'التبريرات': 17904,
 'موارد': 8568,
 'بلدى': 19825,
 'لمحات': 22772,
 'ولدينا': 24686,
 'قابلة': 15412,
 'تعليقاتهم': 7127,
 'سيريني': 11762,
 'اطلقت': 906,
 'الشبه': 13576,
 'الإعلانات': 17631,
 'القواعد': 6922,
 'متشوق': 23006,
 'سانشيز': 8293,
 'وتغيير': 12636,
 'بتطوير': 5641,
 'نصنع': 10158,
 'تسميم': 20228,
 'شودهوري': 21423,
 'لافاييت': 12052,
 'ليبرا': 12210,
 'بقضاء': 14337,
 'شاكر': 21348,
 'الاغذية': 5466,
 'تشاركي': 20236,
 'الخنثى': 18117,
 'افعال': 2356,
 'اكد': 2490,
 'عند': 211,
 'المالك': 13786,
 'عددا': 3247,
 'راقص': 20942,
 'ثاني': 3033,
 'النقابات': 5597,
 'عديم': 11867,
 'انتعاش': 19134,
 'بالتحديد': 7040,
 'ينسوا': 25192,
 'اتفقنا': 12964,
 'قفص': 21927,
 'وحدتنا': 24440,
 'أضيع': 17057,
 

In [18]:
class TranslationDataset(Dataset):
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.src = df["source"].tolist()
        self.trg = df["target"].tolist()


    def __getitem__(self, idx):
        source = self.src[idx]
        target = self.trg[idx]
        return (source, target)

    def __len__(self):
        return len(self.df)

In [19]:
train_dataset = TranslationDataset(df=train)
test_dataset = TranslationDataset(df=test)

In [20]:
def prepare_batch(batch, ar_tokenizer: Tokenizer, en_tokenizer: Tokenizer):
    ar = list(map(lambda x: x[0], batch))
    en = list(map(lambda x: x[1], batch))


    ar = ar_tokenizer.encode_batch(ar)
    en = en_tokenizer.encode_batch(en)


    ar = list(map(lambda x: x.ids, ar))
    en = list(map(lambda x: x.ids, en))


    source = torch.tensor(ar)
    target = torch.tensor(en)
    return (source, target)

In [21]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=16,
                              collate_fn=lambda x: prepare_batch(x, ar_tokenizer, en_tokenizer)
                             )

test_dataloader = DataLoader(dataset=test_dataset, batch_size=16,
                             collate_fn=lambda x: prepare_batch(x, ar_tokenizer, en_tokenizer)
                            )
sample = next(iter(train_dataloader))
torch.count_nonzero(sample[0], dim=1)

tensor([ 9, 12,  4,  5,  5,  8, 15,  7, 38,  9,  6, 12, 12,  6,  6,  6])

In [22]:
class Encoder(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int, latent_dim: int, dropout: float = 0.3):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.latent_dim = latent_dim
        self.dropout_ratio = dropout

        self.embbeding = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.embedding_dim)
        self.rnn = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.latent_dim,
            batch_first=True,
        )

        self.dropout = nn.Dropout(p=self.dropout_ratio)


    def forward(self, x):
        # x = [batch_size, seq_len]

        embeddings = self.dropout(self.embbeding(x))
        # embeddings = [batch_size, seq_len, embedding_dim]


        outputs, (hidden, cell) = self.rnn(embeddings)
        # outputs = [batch_size, seq_len, latent_dim]
        # hidden / cell = [1, batch_size, latent_dim]

        return (hidden, cell)

In [23]:
class Decoder(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim:int, latent_dim: int, dropout: float=0.3):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.latent_dim = latent_dim
        self.dropout_ratio = dropout

        self.embbeding = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.embedding_dim)
        self.rnn = nn.LSTM(input_size=self.embedding_dim, hidden_size=self.latent_dim, batch_first=True)

        self.fc1 = nn.Linear(in_features=self.latent_dim, out_features=self.latent_dim * 2)
        self.fc2 = nn.Linear(in_features=self.latent_dim * 2, out_features=self.vocab_size)

        self.dropout = nn.Dropout(p=self.dropout_ratio)

    # This forward method is only for one step
    def forward(self, x, hidden, cell):
        # x = [batch_size] the input token for this step for all samples in the batch
        # hidden / cell = [1, batch_size, latent_dim]

        embeddings = self.dropout(self.embbeding(x))
        # embeddings = [batch_size, embedding_dim]

        embeddings = embeddings.unsqueeze(1)
        # embeddings = [batch_size, 1, embedding_dim]

        output, (hidden, cell) = self.rnn(embeddings, (hidden, cell))
        # output = [batch_size, 1, latent_dim]
        output = output.squeeze(1)
        # output = [batch_size, latent_dim]

        output = torch.tanh(self.fc1(output))

        output = self.dropout(output)

        output = self.fc2(output)
        # output = [batch_size, vocab_size]

        return output, (hidden, cell)

In [24]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder: nn.Module, decoder: nn.Module, device: torch.device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device


    def forward(self, src: torch.Tensor, max_len: int, start_token: int):
        # src = [batch_size, src_len]
        batch_size = src.shape[0]
        out_vocab_size = self.decoder.vocab_size

        outputs = torch.zeros(batch_size, max_len, out_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)
        inputs = torch.tensor(start_token).repeat(batch_size).to(device)

        for step in range(1, max_len):
            output, (hidden, cell) = self.decoder(inputs, hidden, cell)
            # output = [batch_size, vocab_size]

            outputs[:, step] = output

            top_prediction = output.argmax(dim=1)
            # top_prediction = [batch_size]
            inputs = top_prediction

        return outputs

In [26]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [27]:
EMBEDDING_DIM = 1024
LATENT_DIM = 1024
DROPOUT = 0.5
encoder = Encoder(
    vocab_size=ar_tokenizer.get_vocab_size(),
    embedding_dim=EMBEDDING_DIM,
    latent_dim=LATENT_DIM,
    dropout=DROPOUT
)

decoder = Decoder(
    vocab_size=en_tokenizer.get_vocab_size(),
    embedding_dim=EMBEDDING_DIM,
    latent_dim=LATENT_DIM,
    dropout=DROPOUT
)
model = Seq2Seq(encoder=encoder, decoder=decoder, device=device).to(device)

In [ ]:
optim = torch.optim.Adam(params=model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=0)
for epoch in range(10):
    epoch_train_loss = 0
    epoch_val_loss = 0

    model.train()
    for batch in tqdm(train_dataloader):
        optim.zero_grad()

        src, trg = batch
        src, trg = src.to(device), trg.to(device)

        # src = [batch_size, src_seq_length]
        # trg = [batch_size, trg_seq_length]

        trg_max_len = trg.shape[1]

        output = model(src, trg_max_len, en_tokenizer.token_to_id("[SOS]"))
        output_dim = output.shape[-1]

        output = output[1:].reshape(-1, output_dim)
        trg = trg[1:].reshape(-1)


        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]

        loss = criterion(output, trg)

        loss.backward()



        optim.step()

        epoch_train_loss += loss.item()
    model.eval()
    for batch in tqdm(test_dataloader):
        src, trg = batch
        src, trg = src.to(device), trg.to(device)

        trg_max_len = trg.shape[1]

        output = model(src, trg_max_len, en_tokenizer.token_to_id("[SOS]"))

        # trg = [batch size, trg_max_len]
        # output = [batch size, trg_max_len, output dim]

        output_dim = output.shape[-1]

        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)

        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]

        loss = criterion(output, trg)


        epoch_val_loss += loss.item()


    epoch_train_loss = epoch_train_loss / len(train_dataloader)
    epoch_val_loss = epoch_val_loss / len(test_dataloader)
    print(f"Epoch: {epoch}\tTrain Loss = {epoch_train_loss}\tValidation Loss = {epoch_val_loss}")

  0%|          | 0/1155 [00:00<?, ?it/s]